In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping

# Configuration
SEQ_LENGTH = 10  # Use last 10 draws
NUM_FEATURES = 7  # W1-W6 + WB
MAX_NUM = 49  # Lotto 6/49 max number
EPOCHS = 300  
FUTURE_STEPS = 3  # Predict 3 future draws

# Load and prepare data
df = pd.read_csv("lotto649.csv")  # Load the dataset
data = df.iloc[:, 1:].values  # Exclude date column

# Normalize data
data_normalized = data / MAX_NUM

# Function to create sequences
def create_sequences(data):
    X, y = [], []
    for i in range(len(data) - SEQ_LENGTH):
        X.append(data[i:i + SEQ_LENGTH])
        y.append(data[i + SEQ_LENGTH])
    return np.array(X), np.array(y)

# Create sequences
X, y = create_sequences(data_normalized)

# Build LSTM model
def build_model():
    model = Sequential([
        LSTM(128, return_sequences=True, input_shape=(SEQ_LENGTH, NUM_FEATURES)),
        LSTM(128),
        Dense(128, activation='relu'),
        Dense(NUM_FEATURES, activation='linear')
    ])
    model.compile(optimizer='adam', loss='mae')
    return model

# Train model on all data
model = build_model()
model.fit(X, y, epochs=EPOCHS, batch_size=16, 
          callbacks=[EarlyStopping(patience=30)], verbose=0)

# Use last sequence to predict future draws
last_sequence = X[-1]
future_predictions = []

for i in range(FUTURE_STEPS):
    pred = model.predict(last_sequence.reshape(1, SEQ_LENGTH, NUM_FEATURES))
    pred_denormalized = np.round(pred.reshape(NUM_FEATURES) * MAX_NUM).astype(int)

    # Add randomness to differentiate predictions
    perturbation = np.random.randint(-3, 4, size=NUM_FEATURES)
    pred_denormalized += perturbation
    pred_denormalized = np.clip(pred_denormalized, 1, MAX_NUM)

    future_predictions.append(pred_denormalized)
    last_sequence = np.vstack([last_sequence[1:], pred / MAX_NUM])  # Update sequence

# Print the future draws with labels
labels = ["Best", "Better", "Good"]
print("\nFuture Predictions:")
for i, draw in enumerate(future_predictions):
    print(f"{labels[i]}: {draw}")


FileNotFoundError: [Errno 2] No such file or directory: 'lotto649.csv'